In [2]:
# Importing all the dependencies
import os
from dotenv import load_dotenv
import streamlit as st
import pandas as pd
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader ,TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import RetrievalQA

In [3]:
# Load environment variables from a .env file
os.environ['OPENAI_API_KEY'] =  "sk-proj-n9XgDCkrc5Uiwii2imllT3BlbkFJuGuUeoTXSsiGdmrl0fbh"

In [4]:
# Load the pdf files and extract text
path = "athina_doc.pdf"

loader = PyPDFLoader(path)
documents = loader.load()

In [6]:
# Creating chunks of size 1000 from the extracted text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)

In [7]:
# Creating vector embeddings for the chunks and storing them in the vector store
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(all_splits, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
# System prompt template to guide the model's response behavior
system_template = """You are an AI assistant tasked with answering questions based on the provided context information. If you don't know the answer, just say 'Please ask me relevant questions!', don't try to make up an answer. Keep the answer short and to the point and do not add text that is not asked. Always say "thanks for asking!" at the end of the answer.
            {context}
            Question: {question}
            Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(system_template)

In [9]:
 # Initializing the GPT-4 model
model = ChatOpenAI(model='gpt-4', max_tokens=1000)

# Setting up the retriever with the vector store
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
# Setting up memory to keep track of the conversation history
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [11]:
# Creating the conversational retrieval chain
chain = ConversationalRetrievalChain.from_llm(
        llm=model,
        retriever=retriever,
        memory=memory,
        chain_type='stuff',
        combine_docs_chain_kwargs={'prompt': QA_CHAIN_PROMPT},
        verbose=False
                    )

In [ ]:
# Loading the test_dataset to get the LLM_OUTPUTS
df = pd.read_csv('/datasets/test_dataset.csv')

In [ ]:
# Iterate over all the questions in the test_dataset and get the LLM_OUTPUTS
results = []

for index, row in df.iterrows():
    question = row['Question']
    answer = row['Answer']
    result = chain.invoke(question)
    context = retriever.get_relevant_documents(question)

    results.append({'question': question, 'ground_truth':answer, 'llm_output': result['answer'],'context': context['context']})

# Store the combined data ( Question, Ground Truth, LLM Output, Context) in a .csv file
contexts = pd.DataFrame(results)
contexts.to_csv('results.csv', index=False)

print("Results saved to results.csv")

In [ ]:
results

[{'question': "What is the term for modifications made to a car's standard specifications, including optional extras, by a previous owner?",
  'LLMoutput': 'The term "modifications" refers to any changes to a car\'s standard specifications, including optional extras, whether made by you or a previous owner. These modifications can affect the appearance or performance of the car, such as changes to wheels, suspension, bodywork, and engine among others. Thanks for asking!'},
 {'question': 'What should you do if your car keys are lost or stolen and you need them replaced, considering theft and accidental damage?',
  'LLMoutput': "If your car keys are lost or damaged, the insurance company will either repair or replace them and the locks they fit, including any locksmith charges. You'll need to pay the accidental damage excess. If your keys are stolen, they'll replace the keys and locks, but you'll need to pay the theft excess and report the theft to the police, providing the insurance com